In [1]:
import os
import logging
from datetime import datetime
import pyspark
from pyspark.sql import SparkSession
from datetime import datetime, date
import pandas as pd
from pyspark.sql import Row
from tqdm import tqdm
import boto3

In [21]:
AWS_ENDPOINT = "http://minio-webserver:9000"
AWS_ACCESS_KEY_ID = "U6dHDkvTv3CdrviA"
AWS_SECRET_ACCESS_KEY = "o8LTVjrVtNuzOv7DjGOxhN0HQDeksEej"
AWS_REGION ="us-east-1"

SPARK_APP_NAME= "fiec-antaq-notebook"
SPARK_MASTER_URL="spark://spark-master:7077"

In [16]:
minio_client = boto3.client('s3',
    endpoint_url='http://minio-webserver:9000',
    aws_access_key_id='U6dHDkvTv3CdrviA',
    aws_secret_access_key='o8LTVjrVtNuzOv7DjGOxhN0HQDeksEej',
    aws_session_token=None,
    config=boto3.session.Config(signature_version='s3v4'),
    verify=False,
    region_name='us-east-1'
)
minio_client

In [17]:
def obterJars():
    dir = "./spark-jars"
    jars = os.listdir(dir)
    stringJars = ""

    for jar in jars:
        logging.info(f"{dir}/{jar}")
        stringJars += f"{dir}/{jar},"
    print(stringJars)
    return stringJars[:-1]

In [20]:
# spark_client.stop()
conf = pyspark.SparkConf()
conf.setAppName(SPARK_APP_NAME)
conf.setMaster(SPARK_MASTER_URL)

conf.set("spark.hadoop.fs.s3a.endpoint", AWS_ENDPOINT) \
        .set("spark.hadoop.fs.s3a.endpoint.region", AWS_REGION) \
        .set("spark.hadoop.fs.s3a.access.key",AWS_ACCESS_KEY_ID) \
        .set("spark.hadoop.fs.s3a.secret.key", AWS_SECRET_ACCESS_KEY) \
        .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
        .set("spark.hadoop.fs.s3a.connection.ssl.enabled", False) \
        .set("spark.hadoop.com.amazonaws.services.s3.enableV2", True) \
        .set("spark.hadoop.fs.s3a.committer.staging.conflict-mode", "replace") \
        .set("spark.hadoop.fs.s3a.fast.upload", True) \
        .set("spark.hadoop.fs.s3a.path.style.access", True) \
        .set("spark.sql.sources.commitProtocolClass",
             "org.apache.spark.sql.execution.datasources.SQLHadoopMapReduceCommitProtocol") \
        .set("spark.jars", obterJars()) \
        .set("spark.driver.memory", "12g")\
        .set("spark.ui.showConsoleProgress", "false")
        # .set("spark.executor.memory", "2g") \
        # .set("spark.cores.max", "12") \
        # .set("spark.hadoop.fs.s3a.committer.name", "magic") \
        # .set("spark.hadoop.fs.s3a.bucket.all.committer.magic.enabled", "true")


sc = pyspark.SparkContext(conf=conf).getOrCreate()
sc.setLogLevel("ERROR")
spark_client = SparkSession(sc)
spark_client

RuntimeError: Java gateway process exited before sending its port number